# Netflix Data Cleaning Script

In [2]:
import pandas as pd

#load the Netflix dataset
df = pd.read_csv('netflix_titles.csv')

#standardize column names (lowercase and underscores for consistency)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

#show the first 5 rows
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
#Objective: split multiple directors in the 'director' column into separate rows so that each director is treated individually for accurate analysis

#step 1: filter out rows where 'director' is not null
df_directors = df[pd.notnull(df['director'])].copy()

#step 2: split the director column where multiple names are separated by commas
df_directors['director'] = df_directors['director'].str.split(', ')

#step 3: explode the list of directors so that each one has its own row
df_directors = df_directors.explode('director').reset_index(drop=True)

#step 4: view the cleaned data
df_directors[['title', 'director']].head()

,title,director
0,Dick Johnson Is Dead,Kirsten Johnson
1,Ganglands,Julien Leclercq
2,Midnight Mass,Mike Flanagan
3,My Little Pony: A New Generation,Robert Cullen
4,My Little Pony: A New Generation,José Luis Ucha


In [6]:
#Objective: split the 'listed_in' column (which contains comma-separated genres) into individual rows for proper genre analysis

#step 1: drop rows where 'listed_in' is null
df_genres = df[pd.notnull(df['listed_in'])].copy()

#step 2: split the 'listed_in' column by comma and space (", ")
df_genres['listed_in'] = df_genres['listed_in'].str.split(', ')

#step 3: explode the list into separate rows — each genre gets its own row
df_genres = df_genres.explode('listed_in').reset_index(drop=True)

#step 4: rename the column for clarity
df_genres.rename(columns={'listed_in': 'genre'}, inplace=True)

#step 5: view sample result
df_genres[['title', 'type', 'genre']].head()

,title,type,genre
0,Dick Johnson Is Dead,Movie,Documentaries
1,Blood & Water,TV Show,International TV Shows
2,Blood & Water,TV Show,TV Dramas
3,Blood & Water,TV Show,TV Mysteries
4,Ganglands,TV Show,Crime TV Shows


In [8]:
#Objective: split multiple countries in the 'country' column into separate rows so that each country is treated individually for accurate geographic analysis

#step 1: apply split only if country is not null
df_countries = df.copy()
df_countries['country'] = df_countries['country'].apply(lambda x: x.split(', ') if pd.notnull(x) else x)

#step 2: explode the list of countries so that each one has its own row
df_countries = df_countries.explode('country').reset_index(drop=True)

#step 3: view the cleaned data
df_countries[['title', 'country']].head()

,title,country
0,Dick Johnson Is Dead,United States
1,Blood & Water,South Africa
2,Ganglands,NaN
3,Jailbirds New Orleans,NaN
4,Kota Factory,India


In [10]:
#Objective: convert the 'date_added' column from string to standard datetime format

#step 1: convert the column to datetime using pandas
df['date_added'] = pd.to_datetime(df['date_added'], format='%B %d, %Y', errors='coerce')

#step 2: extract year and store it in a new column 'year_added'
df['year_added'] = df['date_added'].dt.year.fillna(0).astype(int)

#step 3: preview the new column
df[['title', 'date_added', 'year_added']].head()

,title,date_added,year_added
0,Dick Johnson Is Dead,2021-09-25,2021
1,Blood & Water,2021-09-24,2021
2,Ganglands,2021-09-24,2021
3,Jailbirds New Orleans,2021-09-24,2021
4,Kota Factory,2021-09-24,2021


In [12]:
#Objective: extract numeric values from the 'duration' column to create two separate columns:
#1.'movie_duration_min'– duration in minutes (for Movies)
#2.'tv_show_seasons'– number of seasons (for TV Shows)

#step 1: create a column for movie durations in minutes
df['movie_duration_min'] = df.apply(
    lambda row: int(float(str(row['duration']).split()[0]))
    if pd.notnull(row['duration']) and row['type'] == 'Movie'
    else None,
    axis=1
)

#step 2: create a column for number of seasons in TV shows
df['tv_show_seasons'] = df.apply(
    lambda row: int(float(str(row['duration']).split()[0]))
    if pd.notnull(row['duration']) and row['type'] == 'TV Show'
    else None,
    axis=1
)

#step 3: ensure numeric columns are clean integers (no .0 left)
df['movie_duration_min'] = df['movie_duration_min'].fillna(0).astype(int)
df['tv_show_seasons'] = df['tv_show_seasons'].fillna(0).astype(int)

#step 4: check data types
df.dtypes[['movie_duration_min', 'tv_show_seasons']]

movie_duration_min    int32
tv_show_seasons       int32
dtype: object

In [16]:
#Objective: build the final dataset with all cleaned columns and exploded genres, directors, and countries included

#step 1: drop original 'director', 'listed_in', and 'country' columns to avoid duplication
df_base = df.drop(columns=['director', 'listed_in', 'country'])

#step 2: merge the exploded director column back into the base
df_final = df_base.merge(df_directors[['show_id', 'director']], on='show_id', how='left')

#step 3: merge the exploded genre column into the dataset
df_final = df_final.merge(df_genres[['show_id', 'genre']], on='show_id', how='left')

#step 4: merge the exploded country column into the dataset
df_final = df_final.merge(df_countries[['show_id', 'country']], on='show_id', how='left')

#step 5: drop duplicates (safe for SQL or dashboarding)
df_final.drop_duplicates(inplace=True)

#step 6: preview final dataset
df_final.head(25)

,show_id,type,title,cast,date_added,release_year,rating,duration,description,year_added,movie_duration_min,tv_show_seasons,director,genre,country
0,s1,Movie,Dick Johnson Is Dead,NaN,2021-09-25,2020,PG-13,90 min,"As her father nears the end of his life, filmm...",2021,90,0,Kirsten Johnson,Documentaries,United States
1,s2,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021-09-24,2021,TV-MA,2 Seasons,"After crossing paths at a party, a Cape Town t...",2021,0,2,NaN,International TV Shows,South Africa
2,s2,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021-09-24,2021,TV-MA,2 Seasons,"After crossing paths at a party, a Cape Town t...",2021,0,2,NaN,TV Dramas,South Africa
3,s2,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021-09-24,2021,TV-MA,2 Seasons,"After crossing paths at a party, a Cape Town t...",2021,0,2,NaN,TV Mysteries,South Africa
4,s3,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021-09-24,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,2021,0,1,Julien Leclercq,Crime TV Shows,NaN
5,s3,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021-09-24,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,2021,0,1,Julien Leclercq,International TV Shows,NaN
6,s3,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021-09-24,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,2021,0,1,Julien Leclercq,TV Action & Adventure,NaN
7,s4,TV Show,Jailbirds New Orleans,NaN,2021-09-24,2021,TV-MA,1 Season,"Feuds, flirtations and toilet talk go down amo...",2021,0,1,NaN,Docuseries,NaN
8,s4,TV Show,Jailbirds New Orleans,NaN,2021-09-24,2021,TV-MA,1 Season,"Feuds, flirtations and toilet talk go down amo...",2021,0,1,NaN,Reality TV,NaN
9,s5,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",2021-09-24,2021,TV-MA,2 Seasons,In a city of coaching centers known to train I...,2021,0,2,NaN,International TV Shows,India


In [22]:
#Objective: reorder columns in df_final to match the SQL table structure

#step 1: define the column order as per the SQL CREATE TABLE statement
column_order = [
    'show_id',
    'type',
    'title',
    'director',
    'cast',                # corresponds to 'casts' in your schema
    'country',
    'date_added',
    'release_year',
    'rating',
    'duration',
    'genre',
    'description',
    'movie_duration_min',
    'tv_show_seasons',
    'year_added'
]

#step 2: reorder the DataFrame columns
df_final = df_final[column_order]

#step 3: save the cleaned and reordered DataFrame to CSV
df_final.to_csv('netflix_cleaned.csv', index=False)